### Research question and objective
The user will be prompted to input their research question and objective, from where we would get the necessary keywords and is also used to screen abstracts

## SEARCH PART

Search strategy - Included all relevant papers with keyword extraction, and all the papers are downloaded. Once the papers are retrieved, they go through a screening process using abstracts, rank with respect to relevance.

In [ ]:
!pip install python-dotenv

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Semantic_key.env to Semantic_key.env


In [ ]:
import requests
import os, time
import concurrent.futures
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from dotenv import load_dotenv

# Function to retreive fulltext url / doi url from DOAJ API response
def get_fulltext_from_bibjson(bibjson):
    # First try to get fulltext link from DOAJ metadata
    for link in bibjson.get("link", []):
        if link.get("type") == "fulltext":
            return link.get("url")

    # If full text url not available, build fulltext landing page from DOI (redirects to publisher)
    for id_obj in bibjson.get("identifier", []):
        if id_obj.get("type") == "doi":
            return f"https://doi.org/{id_obj.get('id')}"

    return None

# Function to search DOAJ DB
def doaj_search(query, page_size=10):
    url = f"https://doaj.org/api/v2/search/articles/{query}?pageSize={page_size}"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)

    print(f"[DOAJ] Response status: {response.status_code}")
    papers = []

    if response.status_code == 200:
        data = response.json()
        for result in data.get("results", []):
            bibjson = result.get("bibjson", {})
            title = bibjson.get("title", "N/A")
            abstract = bibjson.get("abstract", "")
            full_text_url = get_fulltext_from_bibjson(bibjson)

            papers.append({
                "source": "DOAJ",
                "title": title,
                "abstract": abstract,
                "full_text_url": full_text_url
            })
    return papers

# Function to search EuropePMC DB
def europe_pmc_search(query, max_results=30):
    params = {"query": query + " + OPEN_ACCESS:Y", "format": "json", "pageSize": max_results}
    url = "https://www.ebi.ac.uk/europepmc/webservices/rest/search"
    response = requests.get(url, params=params)
    print(f"[Europe PMC] Response status: {response.status_code}")
    papers = []
    if response.status_code == 200:
        data = response.json().get("resultList", {}).get("result", [])
        for item in data:
            papers.append({
                "source": "Europe PMC",
                "title": item.get('title', 'N/A'),
                "abstract": item.get('abstractText', ''),
                "link": f"https://europepmc.org/article/{item.get('source', '')}/{item.get('id', '')}",
                "full_text_url": f"https://europepmc.org/backend/ptpmcrender.fcgi?accid={item.get('pmcid', '')}&blobtype=pdf"
            })
    return papers

# Function to search semantic scholar DB
load_dotenv()

def search_semantic_scholar(query, max_results=25):
    url = "https://api.semanticscholar.org/graph/v1/paper/search"

    # Load the API key from the environment variable
    API_KEY = os.getenv("API_KEY")

    headers = {
        "Authorization": f"Bearer {API_KEY}"
    }

    params = {
        "query": query,
        "fields": "title,abstract,url,openAccessPdf",  # Fields you want to retrieve
        "limit": max_results  # Number of results to return
    }

    # Retry logic for rate-limiting (HTTP status 429)
    retries = 10
    for attempt in range(retries):
        response = requests.get(url, headers=headers, params=params)

        # If the request is successful (status 200), process the response
        if response.status_code == 200:
            print(f"[Semantic Scholar] Status: {response.status_code}")
            open_access_papers = []
            results = response.json().get("data", [])
            for paper in results:
                if paper.get("openAccessPdf") and paper["openAccessPdf"].get("url"):
                    open_access_papers.append({
                        "source": "Semantic Scholar",
                        "title": paper.get("title"),
                        "abstract": paper.get("abstract", ""),
                        "full_text_url": paper["openAccessPdf"]["url"]
                    })
            return open_access_papers

        # If rate-limited, back off and retry after a delay
        elif response.status_code == 429:
            print(f"[Semantic Scholar] Rate-limited. Retrying in {2 ** attempt} seconds...")
            time.sleep(2 ** attempt)  # Exponential backoff
        else:
            print(f"[Semantic Scholar] Error: {response.status_code}")
            break  # Exit on other errors like 4xx, 5xx

    # If we exhausted retries and still getting rate-limited
    return []

# Function to perform parallel search on EuropePMC, DOAJ & Semantic Scholar APIs
def parallel_search(query, max_results=30):
    results = {"EUROPEPMC": [], "DOAJ": [], "SEMANTIC" : []}

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_pubmed = executor.submit(europe_pmc_search, query, max_results)
        future_doaj = executor.submit(doaj_search, query, max_results)
        future_semantic = executor.submit(search_semantic_scholar, query, max_results)

        results["EUROPEPMC"] = future_pubmed.result()
        results["DOAJ"] = future_doaj.result()
        results["SEMANTIC"] = future_semantic.result()

    return results

# Function to check if full text url is a direct pdf link
def is_direct_pdf_link(url):
    try:
        response = requests.head(url, headers={"User-Agent": "Mozilla/5.0"}, allow_redirects=True, timeout=10)
        return 'application/pdf' in response.headers.get("Content-Type", "").lower()
    except:
        return False

# Function used to get pdf link from unpaywall if only doi is available
def get_pdf_from_unpaywall(doi, email="your_email@example.com"):
    api_url = f"https://api.unpaywall.org/v2/{doi}?email={email}"
    try:
        response = requests.get(api_url)
        if response.status_code == 200:
            data = response.json()
            oa_location = data.get("best_oa_location")
            if oa_location and oa_location.get("url_for_pdf"):
                return oa_location["url_for_pdf"]
    except:
        pass
    return None

# Function to find pdf link if the full text url is pointing to a webpage instead of direct link
def extract_pdf_link_from_html_page(page_url):
    headers = {"User-Agent": "Mozilla/5.0"}
    try:
        response = requests.get(page_url, headers=headers, timeout=10)
        if response.status_code != 200:
            return None
        soup = BeautifulSoup(response.text, "html.parser")
        for link in soup.find_all("a", href=True):
            classes = link.get("class", [])
            href = link["href"]

            # Match if any class contains 'pdf' or class list includes 'pdf'
            if "pdf" in href.lower() or any("pdf" in cls.lower() for cls in classes):
                full_link = urljoin(page_url, href)
                if full_link.endswith(".pdf") or "view" in full_link:  # customize as needed
                    return full_link
    except:
        pass
    return None

# Function to download the pdf to local if the url is a direct link
def download_pdf_from_url(pdf_url, save_path="paper.pdf"):
    headers = {"User-Agent": "Mozilla/5.0"}
    try:
        response = requests.get(pdf_url, headers=headers, stream=True, timeout=10)
        content_type = response.headers.get("Content-Type", "")

        if response.status_code == 200 and 'application/pdf' in content_type:
            with open(save_path, "wb") as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
            print(f"PDF downloaded: {save_path}")
            return save_path
        else:
            print(f"Skipped (Not a PDF or blocked): {pdf_url} [Content-Type: {content_type}]")
    except Exception as e:
        print(f"Error downloading PDF from {pdf_url}: {e}")
    return None

# Function to rename the downloaded file based on title of the paper
def sanitize_filename(title, version=""):
    return "".join(c if c.isalnum() or c in "._-" else "_" for c in title)[:100] + (f"_{version}" if version else "")

# Function to download the papers after checking if it is a direct link/doi/webpage
def download_all_pdfs(results):
    os.makedirs("downloads", exist_ok=True)
    all_papers = results["EUROPEPMC"] + results["DOAJ"] + results["SEMANTIC"]

    for idx, paper in enumerate(all_papers):
        title = paper.get("title", f"paper_{idx}")
        pdf_url = paper.get("full_text_url")

        if pdf_url:
            filename = sanitize_filename(title) + ".pdf"
            save_path = os.path.join("downloads", filename)
            print(f"\nDownloading Paper {idx + 1}: {title}")
            resolve_pdf_url_and_download(pdf_url, save_path)

# Function to resolve the pdf url correctly based on api response and download the paper accordingly
def resolve_pdf_url_and_download(full_text_url, save_path):

    # Direct PDF check
    if is_direct_pdf_link(full_text_url):
        return download_pdf_from_url(full_text_url, save_path)

    # Check if link is DOI and use Unpaywall to download
    parsed = urlparse(full_text_url)
    if "doi.org" in parsed.netloc:
        doi = parsed.path.strip("/")
        print(f"Using Unpaywall for DOI: {doi}")
        pdf_url = get_pdf_from_unpaywall(doi)
        if pdf_url and is_direct_pdf_link(pdf_url):
            return download_pdf_from_url(pdf_url, save_path)

    # Scrape the webpage to find PDF url and download
    print(f"Scraping HTML for PDF: {full_text_url}")
    pdf_url = extract_pdf_link_from_html_page(full_text_url)
    if pdf_url and is_direct_pdf_link(pdf_url):
        return download_pdf_from_url(pdf_url, save_path)

    print(f"No downloadable PDF found")
    return

print("\n Welcome to the PRISMA-ScR Automated Research Tool!")
query = input("Enter keywords to search for research papers: ")

print("Searching PubMed, DOAJ, SEMANTIC SCHOLAR databases... Please wait...\n")
search_results = parallel_search(query, max_results=30)

download_all_pdfs(search_results)
print("Search complete!")


 Welcome to the PRISMA-ScR Automated Research Tool!
Enter keywords to search for research papers: How does AI improve medical diagnostics
Searching PubMed, DOAJ, SEMANTIC SCHOLAR databases... Please wait...

[Semantic Scholar] Rate-limited. Retrying in 1 seconds...
[DOAJ] Response status: 200
[Europe PMC] Response status: 200
[Semantic Scholar] Rate-limited. Retrying in 2 seconds...
[Semantic Scholar] Status: 200

PDF downloaded: downloads/Rethinking_sepsis__from_controversy_to_precision-driven_solutions..pdf

PDF downloaded: downloads/AI_and_Interventional_Radiology__A_Narrative_Review_of_Reviews_on_Opportunities__Challenges__and_Fut.pdf

PDF downloaded: downloads/Does_Artificial_Intelligence_Bring_New_Insights_in_Diagnosing_Phlebological_Diseases_-A_Systematic_R.pdf

PDF downloaded: downloads/Proceedings_of_the_Clinical_Microbiology_Open_2024__artificial_intelligence_applications_in_clinical.pdf

PDF downloaded: downloads/Artificial_Intelligence_in_Relation_to_Accurate_Information_a

No reference query - How does AI improve medical diagnostics

---

RQ | No of papers in search | Initial versus final ranking (more comprehensive) | F-Score for summaries

## Ensemble model
The code uses multiple language models (Gemini and Mistral) to evaluate research papers based on their abstracts, rank them by relevance to a research question, and generate comprehensive summaries. By combining results from multiple models, we aim to produce more robust and accurate outputs than any single model alone.

#Setup Instructions
To use this code, you'll need:

*   API keys for Gemini and Mistral models

you can use the .env file approach:
```
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('Semantic_key.env')
```
How ensemble model works:



*   Paper Ranking: Each model ranks papers based on their relevance to a research question
*   Ensemble Ranking: Rankings are combined to create a consensus ranking
*   Summarization: The top-ranked papers are summarized individually and collectively
*   Ensemble Summary: Model summaries are synthesized into a final summary
*   Evaluation: Summaries are evaluated using BERT Score against reference summaries


In [ ]:
# LOAD API KEYS TO RUN LLM APIs
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('Semantic_key.env')

# Example format for your Semantic_key.env file:
# GEMINI_API_KEY=your_gemini_api_key_here
# MISTRAL_API_KEY=your_mistral_api_key_here

# Test that keys are loaded properly
print("API keys loaded successfully" if os.environ.get("GEMINI_API_KEY") and os.environ.get("MISTRAL_API_KEY") else "Failed to load API keys")

In [ ]:
!pip install bert-score mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.5/301.5 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

In [ ]:
import numpy as np
import json
import requests
import torch
import os
from google import genai
from bert_score import score as bert_score
from mistralai import Mistral

def query_gemini(prompt, api_key):
    """
    Send a prompt to the Gemini API and get a text response

    Args:
        prompt (str): The text prompt to send to Gemini
        api_key (str): The Gemini API key

    Returns:
        str or None: The text response from Gemini, or None if there was an error
    """
    client = genai.Client(api_key=api_key)
    try:
        response = client.models.generate_content(
          model="gemini-2.0-flash", contents=prompt
        )
        return response.text
    except Exception as e:
        print(f"Error querying Gemini API: {e}")
        return None

def query_mistral(prompt, api_token):
    """
    Send a prompt to the Mistral API and get a response (attempts to parse as JSON)

    Args:
        prompt (str): The text prompt to send to Mistral
        api_token (str): The Mistral API token

    Returns:
        dict or str or None: JSON response if valid, raw text otherwise, or None if there was an error
    """
    try:
        model = "mistral-small-latest"
        print("Sending prompt to Mistral API...")
        client = Mistral(api_key=api_token)
        chat_response = client.chat.complete(
            model=model,
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ]
        )

        response_content = chat_response.choices[0].message.content

        # Try to parse as JSON
        try:
            json_response = json.loads(response_content)
            return json_response
        except json.JSONDecodeError:
            print("Warning: Response is not valid JSON. Returning raw text.")
            return response_content

    except Exception as e:
        print(f"Error querying Mistral API: {e}")
        return None

def parse_ranking_response(response_text):
    """
    Parse the ranking response from a language model

    Args:
        response_text (str): The raw text response from the model

    Returns:
        tuple: (ranking list, scores list) or (None, []) if parsing failed
    """
    try:
        # Try to extract JSON from the response
        start_idx = response_text.find('{')
        end_idx = response_text.rfind('}')

        if start_idx >= 0 and end_idx > start_idx:
            json_str = response_text[start_idx:end_idx+1]
            ranking_data = json.loads(json_str)

            if isinstance(ranking_data, dict) and "ranking" in ranking_data:
                return ranking_data["ranking"], ranking_data.get("scores", [])

        # Fallback to parsing numbered list
        lines = response_text.strip().split('\n')
        ranking = []

        for line in lines:
            if line and ':' in line:
                parts = line.split(':', 1)
                try:
                    index = int(parts[0].strip().rstrip('.')) - 1
                    ranking.append(index)
                except ValueError:
                    pass

        return ranking, [] if ranking else None, []
    except Exception as e:
        print(f"Error parsing ranking response: {e}")
        return None, []

def get_model_rankings(abstracts, titles, query, gemini_api_key, hf_api_token):
    """
    Get rankings of abstracts from both Gemini and Mistral models

    Args:
        abstracts (list): List of paper abstracts
        titles (list): List of paper titles
        query (str): The research question
        gemini_api_key (str): Gemini API key
        hf_api_token (str): Mistral API token

    Returns:
        dict: Dictionary containing rankings and scores from both models
    """
    abstract_list = ""
    for i, abs in enumerate(abstracts):
        abstract_list += f"{i+1}. {abs}\n"  # If abs is a string. Adjust if dict.

    prompt = f"""
      You are a research assistant helping with a scoping review.

      RESEARCH QUESTION: {query}

      I have retrieved the following {len(abstracts)} papers. Please rank them based on their relevance to the research question:

      {abstract_list}

      Based on their abstracts, rank these papers in order of relevance to my research question.
      For each paper, assign a relevance score from 0 to 10, where 10 is most relevant.

      Return your answer as a JSON object with the following format:
      {{
        "ranking": [list of paper numbers in order of relevance from most to least relevant],
        "scores": [corresponding relevance scores for each paper]
      }}

      Provide ONLY the JSON response with no additional text.
      """
    gemini_response = query_gemini(prompt, gemini_api_key)
    mistral_response = query_mistral(prompt, hf_api_token)
    gemini_ranking, gemini_scores = parse_ranking_response(gemini_response) if gemini_response else (None, [])
    mistral_ranking, mistral_scores = parse_ranking_response(mistral_response) if mistral_response else (None, [])

    results = {
        "gemini": {
            "ranking": gemini_ranking if gemini_ranking else [],
            "scores": gemini_scores if gemini_scores else []
        },
        "mistral": {
            "ranking": mistral_ranking if mistral_ranking else [],
            "scores": mistral_scores if mistral_scores else []
        }
    }

    return results

def get_ensemble_ranking(abstracts, titles, query, results_from_models=None):
    """
    Create an ensemble ranking by combining rankings from multiple models

    Args:
        abstracts (list): List of paper abstracts
        titles (list): List of paper titles
        query (str): The research question
        results_from_models (dict, optional): Pre-computed model rankings

    Returns:
        tuple: (ensemble_ranked, ensemble_scores) where ensemble_ranked is an array of indices
               ranked by relevance and ensemble_scores are their corresponding scores
    """
    if not abstracts:
        return np.arange(len(abstracts)), np.zeros(len(abstracts))

    if results_from_models is None:
        # Get API keys from environment variables
        gemini_api_key = os.environ.get("GEMINI_API_KEY")
        hf_api_token = os.environ.get("MISTRAL_API_KEY")
        results_from_models = get_model_rankings(abstracts, titles, query, gemini_api_key, hf_api_token)

    gemini_ranking = np.array(results_from_models["gemini"]["ranking"]) if results_from_models["gemini"]["ranking"] else np.arange(len(abstracts))
    mistral_ranking = np.array(results_from_models["mistral"]["ranking"]) if results_from_models["mistral"]["ranking"] else np.arange(len(abstracts))

    # Ensure rankings are the correct length
    if len(gemini_ranking) != len(abstracts):
        gemini_ranking = np.arange(len(abstracts))
    if len(mistral_ranking) != len(abstracts):
        mistral_ranking = np.arange(len(abstracts))

    # Calculate ensemble scores based on average position in the rankings
    ensemble_scores = np.zeros(len(abstracts))
    for i in range(len(abstracts)):
        gemini_position = np.where(gemini_ranking == i)[0][0] if i in gemini_ranking else len(abstracts)
        mistral_position = np.where(mistral_ranking == i)[0][0] if i in mistral_ranking else len(abstracts)

        # Use inverse of average position as the score (lower position = higher score)
        ensemble_scores[i] = 1 / (1 + (gemini_position + mistral_position) / 2)

    # Sort by score in descending order
    ensemble_ranked = np.argsort(ensemble_scores)[::-1]

    return ensemble_ranked, ensemble_scores

def get_model_summaries(abstracts, titles, ranking, query, gemini_api_key, hf_api_token, top_n=5):
    """
    Generate summaries from both models for the top-ranked papers

    Args:
        abstracts (list): List of paper abstracts
        titles (list): List of paper titles
        ranking (list): List of paper indices in order of relevance
        query (str): The research question
        gemini_api_key (str): Gemini API key
        hf_api_token (str): Mistral API token
        top_n (int): Number of top papers to include in the summary

    Returns:
        dict: Dictionary containing summaries from both models
    """
    # Get the top n paper indices
    top_indices = ranking[:min(top_n, len(ranking))]

    # Format paper information for the prompt
    papers_info = []
    for i, idx in enumerate(top_indices):
        paper_info = f"{i+1}: \"{titles[idx]}\"\n"
        paper_info += f"Abstract: {abstracts[idx]}\n"
        papers_info.append(paper_info)

    papers_text = "\n\n".join(papers_info)

    # Create the prompt for the summaries
    prompt = f"""
    You are a research assistant helping with a scoping review, following PRISMA Guidelines.
    RESEARCH QUESTION: {query}
    Based on the top {len(top_indices)} papers below, create a comprehensive summary that:
    1. Identifies key themes and findings across the papers
    2. Highlights methodological approaches used
    3. Notes any gaps in the literature
    4. Suggests directions for future research
    Papers:
    {papers_text}
    Provide ONLY a well-structured summary that synthesizes the information from these papers with no additional text.
    """

    # Get summaries from both models
    gemini_summary = query_gemini(prompt, gemini_api_key)
    mistral_summary = query_mistral(prompt, hf_api_token)

    return {
        "gemini": gemini_summary,
        "mistral": mistral_summary
    }

def get_ensemble_summaries(abstracts, titles, query, results_from_models=None):
    """
    Create an ensemble summary by combining summaries from multiple models

    Args:
        abstracts (list): List of paper abstracts
        titles (list): List of paper titles
        query (str): The research question
        results_from_models (dict, optional): Pre-computed model rankings

    Returns:
        str: The ensemble summary text
    """
    if not abstracts:
        return "No abstracts provided for summarization."

    # Get the ensemble ranking
    ensemble_ranked, _ = get_ensemble_ranking(abstracts, titles, query, results_from_models)

    # Get API keys from environment variables
    gemini_api_key = os.environ.get("GEMINI_API_KEY")
    hf_api_token = os.environ.get("MISTRAL_API_KEY")

    # Get summaries from both models
    summaries = get_model_summaries(abstracts, titles, ensemble_ranked, query, gemini_api_key, hf_api_token)

    # Create an ensemble summary if both model summaries exist
    if summaries["gemini"] and summaries["mistral"]:
        ensemble_prompt = f"""
        You are a research assistant helping with a scoping review, following PRISMA guidelines.

        I have two summaries of the same set of papers related to this research question: "{query}"

        Summary 1:
        {summaries["gemini"]}

        Summary 2:
        {summaries["mistral"]}

        Please create a synthesis of these two summaries, incorporating the strongest insights and analysis from each.
        The final summary should be comprehensive yet concise, highlighting key themes, methods, gaps, and future directions.
        Do not reference these summaries and just output the final summary.
        """

        ensemble_summary = query_gemini(ensemble_prompt, gemini_api_key)
        return ensemble_summary
    elif summaries["gemini"]:
        return summaries["gemini"]
    elif summaries["mistral"]:
        return summaries["mistral"]
    else:
        return "Unable to generate summaries from the provided models."

def get_model_summaries_for_each_paper(abstracts, titles, query, gemini_api_key, hf_api_token):
    """
    Generate individual summaries for each paper from both models

    Args:
        abstracts (list): List of paper abstracts
        titles (list): List of paper titles
        query (str): The research question
        gemini_api_key (str): Gemini API key
        hf_api_token (str): Mistral API token

    Returns:
        dict: Dictionary containing combined summaries from both models
    """
    gemini_summary = ""
    mistral_summary = ""

    for i in range(len(abstracts)):
      prompt = f"""
      Summarize this research paper: Title - {titles[i]}, abstract - {abstracts[i]}
      Provide ONLY a well-structured summary that synthesizes the information from these papers with no additional text.
      """

      gemini_summary += (query_gemini(prompt, gemini_api_key))
      mistral_summary +=  (query_mistral(prompt, hf_api_token))
      time.sleep(1)  # Add a small delay to avoid rate limiting

    return {
        "gemini_ep": gemini_summary,
        "mistral_ep": mistral_summary
    }

def get_ensemble_summaries_for_each_paper(abstracts, titles, query, results_from_models=None):
    """
    Create an ensemble of individual paper summaries from multiple models

    Args:
        abstracts (list): List of paper abstracts
        titles (list): List of paper titles
        query (str): The research question
        results_from_models (dict, optional): Pre-computed model rankings

    Returns:
        str: The ensemble summary text combining individual paper summaries
    """
    if not abstracts:
        return "No abstracts provided for summarization."

    # Get API keys from environment variables
    gemini_api_key = os.environ.get("GEMINI_API_KEY")
    hf_api_token = os.environ.get("MISTRAL_API_KEY")

    # Get individual paper summaries from both models
    summaries = get_model_summaries_for_each_paper(abstracts, titles, query, gemini_api_key, hf_api_token)

    # Create an ensemble summary if both model summaries exist
    if summaries["gemini_ep"] and summaries["mistral_ep"]:
        ensemble_prompt = f"""
        I have two summaries of the same set of papers related to this research question: "{query}"

        Summary 1:
        {summaries["gemini_ep"]}

        Summary 2:
        {summaries["mistral_ep"]}

        Please create a synthesis of these two summaries maintaining academic standard. Do not reference these summaries and just output the final summary.
        """

        ensemble_summary = query_gemini(ensemble_prompt, gemini_api_key)
        return ensemble_summary
    elif summaries["gemini_ep"]:
        return summaries["gemini_ep"]
    elif summaries["mistral_ep"]:
        return summaries["mistral_ep"]
    else:
        return "Unable to generate summaries from the provided models."

def evaluate_summaries_with_bert(summaries, reference_summary):
    """
    Evaluate summaries using BERT score against a reference summary

    Args:
        summaries (dict): Dictionary of model names to summary texts
        reference_summary (str): The reference summary to compare against

    Returns:
        dict: Dictionary of model names to score dictionaries (precision, recall, f1)
    """
    scores = {}

    # Evaluate each model's summary
    for model_name, summary in summaries.items():
        if summary:
            try:
                P, R, F1 = bert_score([summary], [reference_summary], lang="en", rescale_with_baseline=True)
                scores[model_name] = {
                    "precision": P.item(),
                    "recall": R.item(),
                    "f1": F1.item()
                }
            except Exception as e:
                print(f"Error computing BERT Score for {model_name}: {e}")
                scores[model_name] = None

    # If we have an ensemble summary, evaluate it too
    if "ensemble" in summaries and summaries["ensemble"]:
        try:
            P, R, F1 = bert_score([summaries["ensemble"]], [reference_summary], lang="en", rescale_with_baseline=True)
            scores["ensemble"] = {
                "precision": P.item(),
                "recall": R.item(),
                "f1": F1.item()
            }
        except Exception as e:
            print(f"Error computing BERT Score for ensemble: {e}")
            scores["ensemble"] = None

    return scores

## Abstract Screening

The abstracts of the relevant papers go through screening process below. Screening strategy is used by ranking the papers with sentence transformers and ensemble model. All the ranks obtained by each paper will have a final consolidated ranking from the models to minimise the model bias using Reciprocal Rank Fusion

### Sentence Transformers used
The following pretrained sentence transformers are used:
* BM25
* SBERT
* SPLADE

and Ensemble model of Deepseek and Gemini LLMs.

The abstracts are ranked on the relevance similarity scores based on mean consolidated embeddings with other abstracts and research question. Say, we have 'N' total papers retrieved and we use the above models(m ∈ [1,4]),\
<center>$R_{abs_i}^{model_m} = SS_{model_m}(abs_i, \frac{1}{2N}(Σ_{j!=i}abs_j)+rq/2)$</center>, where,\

* R_{abs_i}^{model_m} is the rank of paper i with respect to model m,
* SS_{model_m}(a, b) is similarity score with respect to model m between a and b, a and b are two text embedding vectors,
* abs_i is abstract embedding vector of paper i,
* rq is embedding vector of research question.

Now, we have m ranks for each paper, which can possibly include model bias because they are trained over different kinds of data.

To reduce this bias, we adopt RRF:
<center> $Rank_{abs_i} = Σ_{j=1}^m \frac{1}{k+R_{abs_i}^{model_j} }$</center>
where,


* Rank_{abs_i} is the final rank of i^{th} paper,
* k is a constant, generally used 60
* R_{abs_i}^{model_j} is the rank of i^{th} paper with respect to model j.



In [ ]:
def get_abstracts_from_papers(results):
    """
    Extract abstracts from the search results
    """
    all_papers = results["EUROPEPMC"] + results["DOAJ"] + results["SEMANTIC"]
    abstracts = []
    titles = []

    for paper in all_papers:
        title = paper.get("title", "")
        abstract = paper.get("abstract", "")

        if abstract is not None and abstract.strip():  # Only include papers with non-empty abstracts
            abstracts.append(abstract)
            titles.append(title)

    return abstracts, titles

In [ ]:
!pip install rank_bm25

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
import torch
import os
import json

# Reciprocal Rank Fusion (RRF) Function
def reciprocal_rank_fusion(ranked_lists, k=60):
    scores = {}
    for rank_list in ranked_lists:
        for rank, doc_id in enumerate(rank_list):
            scores[doc_id] = scores.get(doc_id, 0) + 1 / (k + rank + 1)
    return sorted(scores.items(), key=lambda x: x[1], reverse=True)

# BM25 Ranking
def get_bm25_ranking(abstracts, query):
    tokenized_abstracts = [doc.split() for doc in abstracts]
    bm25 = BM25Okapi(tokenized_abstracts)
    bm25_scores = bm25.get_scores(query.split())
    return np.argsort(bm25_scores)[::-1], bm25_scores

# SBERT Ranking
def get_sbert_ranking(abstracts, query):
    sbert_model = SentenceTransformer('all-MiniLM-L6-v2')
    abstract_embeddings = sbert_model.encode(abstracts, convert_to_tensor=True)
    query_embedding = sbert_model.encode([query], convert_to_tensor=True)

    # Calculate mean abstract embedding to use in similarity calculation
    mean_abstract_embedding = torch.mean(abstract_embeddings, dim=0, keepdim=True)
    # Combine mean abstract and research question as described in the formula
    combined_embedding = 0.5 * mean_abstract_embedding + 0.5 * query_embedding

    # Calculate similarity scores for each abstract with the combined embedding
    abstract_embeddings = abstract_embeddings.cpu().numpy()
    combined_embedding = combined_embedding.cpu().numpy()
    sbert_scores = cosine_similarity(abstract_embeddings, combined_embedding).flatten()

    return np.argsort(sbert_scores)[::-1], sbert_scores

# SPLADE Ranking
def get_splade_ranking(abstracts, query):
    try:
        splade_tokenizer = AutoTokenizer.from_pretrained("naver/splade-cocondenser-ensembledistil")
        splade_model = AutoModel.from_pretrained("naver/splade-cocondenser-ensembledistil")

        def get_splade_representation(text):
            inputs = splade_tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
            with torch.no_grad():
                outputs = splade_model(**inputs).last_hidden_state.mean(dim=1)
            return outputs.squeeze().cpu().numpy()

        splade_embeddings = np.array([get_splade_representation(text) for text in abstracts])
        query_splade_embedding = get_splade_representation(query)

        # Calculate mean abstract embedding
        mean_splade_embedding = np.mean(splade_embeddings, axis=0)
        # Combine mean abstract and research question
        combined_embedding = 0.5 * mean_splade_embedding + 0.5 * query_splade_embedding

        splade_scores = cosine_similarity(splade_embeddings, combined_embedding.reshape(1, -1)).flatten()
        return np.argsort(splade_scores)[::-1], splade_scores
    except Exception as e:
        print(f"Error in SPLADE ranking: {e}")
        # Return dummy ranking if SPLADE fails
        return np.arange(len(abstracts)), np.zeros(len(abstracts))

def printRankings(ranked_results):
  for i in ranked_results:
    print(f'Rank: {i["rank"]}       Title:{i["title"]}       Relevance Score:{i["relevance_score"]}')
    print("\n")

def rank_abstracts(search_results, research_question, rfm=None):
    # Extract abstracts from search results
    abstracts, titles = get_abstracts_from_papers(search_results)

    if not abstracts:
        print("No abstracts found in the search results")
        return []

    print(f"Ranking {len(abstracts)} abstracts based on relevance to research question...")

    # Get rankings from each model
    bm25_ranked, bm25_scores = get_bm25_ranking(abstracts, research_question)
    sbert_ranked, sbert_scores = get_sbert_ranking(abstracts, research_question)
    splade_ranked, splade_scores = get_splade_ranking(abstracts, research_question)

    # Collect results from the first three models to use in ensemble
    results_from_models = {
        "bm25": {"ranking": bm25_ranked.tolist(), "scores": bm25_scores.tolist()},
        "sbert": {"ranking": sbert_ranked.tolist(), "scores": sbert_scores.tolist()},
        "splade": {"ranking": splade_ranked.tolist(), "scores": splade_scores.tolist()}
    }

    # Get ensemble model ranking
    ensemble_ranked, ensemble_scores = get_ensemble_ranking(abstracts, titles, research_question, rfm)

    # Apply RRF to Combine Rankings
    ranked_lists = [bm25_ranked, sbert_ranked, splade_ranked, ensemble_ranked]
    final_ranking = reciprocal_rank_fusion(ranked_lists)

    # Create the final ranked results
    ranked_results = []
    for idx, (doc_id, score) in enumerate(final_ranking):
        if doc_id < len(titles):  # Ensure valid index
            ranked_results.append({
                "rank": idx + 1,
                "title": titles[doc_id],
                "abstract": abstracts[doc_id],
                "relevance_score": score
            })

    # Save rankings for later use
    os.makedirs("results", exist_ok=True)
    with open("results/abstract_rankings.json", "w") as f:
        json.dump(ranked_results, f, indent=2)

    return ranked_results

Ranking the abstracts

In [ ]:
# ----------------CODE TO RUN THE PIPELINE----------------------#
def run_review_pipeline(query, abstracts, titles, reference_summary=None):

    gemini_api_key = os.environ.get("GEMINI_API_KEY")
    hf_api_token = os.environ.get("MISTRAL_API_KEY")

    print("Getting model rankings...")
    model_rankings = get_model_rankings(abstracts, titles, query, gemini_api_key, hf_api_token)

    print("Creating ensemble ranking...")
    ensemble_ranked, ensemble_scores = get_ensemble_ranking(abstracts, titles, query, model_rankings)

    print("Final ranking...")
    final_ranking = rank_abstracts(search_results, query, model_rankings)

    print("Generating model summaries...")
    model_summaries = get_model_summaries(abstracts, titles, ensemble_ranked, query, gemini_api_key, hf_api_token)

    print("Creating ensemble summary...")
    ensemble_summary = get_ensemble_summaries(abstracts, titles, query, model_rankings)

    print("Generating model summaries for each paper...")
    model_summaries_ep = get_model_summaries_for_each_paper(abstracts, titles, query, gemini_api_key, hf_api_token)

    print("Creating ensemble summary for each paper..")
    ensemble_summary_ep = get_ensemble_summaries_for_each_paper(abstracts, titles, query, model_rankings)

    all_summaries = {
        "gemini": model_summaries["gemini"],
        "mistral": model_summaries["mistral"],
        "ensemble": ensemble_summary,
        "gemini_ep": model_summaries_ep["gemini_ep"],
        "mistral_ep": model_summaries_ep["mistral_ep"],
        "ensemble_ep": ensemble_summary_ep
    }

    results = {
        "rankings": {
            "gemini": model_rankings["gemini"],
            "mistral": model_rankings["mistral"],
            "ensemble": {
                "ranking": ensemble_ranked.tolist(),
                "scores": ensemble_scores.tolist()
            },
            "final" : final_ranking
        },
        "summaries": all_summaries
    }

    # Step 5: Evaluate with BERT Score if reference is provided
    if reference_summary:
        print("Evaluating summaries with BERT Score...")
        bert_scores = evaluate_summaries_with_bert(all_summaries, reference_summary)
        results["evaluation"] = bert_scores

    return results

In [ ]:
abstracts, titles = get_abstracts_from_papers(search_results)
results = run_review_pipeline(query, abstracts, titles)

# Print rankings
print("\nPaper Rankings:")
print("==============")
print("\nGemini Ranking:")
# print("results!!!----------->", results)
for i, idx in enumerate(results["rankings"]["gemini"]["ranking"]):
    print(f"{i+1}. {titles[idx-1]}")

print("\nMistral Ranking:")
for i, idx in enumerate(results["rankings"]["mistral"]["ranking"]):
    print(f"{i+1}. {titles[idx-1]}")

print("\nEnsemble Ranking:")
for i, idx in enumerate(results["rankings"]["ensemble"]["ranking"]):
    print(f"{i+1}. {titles[idx]} (Score: {results['rankings']['ensemble']['scores'][i]:.4f})")

print("\nFinal Ranking:")
for i in (results["rankings"]["final"]):
  print(f"{i['rank']}.{i['title']} (Score: {i['relevance_score']:.4f})")

# Print summaries (truncated for brevity)
print("\nConsolidated Summary:")
print("=================================")
for model, summary in results["summaries"].items():
    if summary:
        print(f"\n{model.capitalize()} Summary: {summary[:]}...")

# Print BERT Score evaluation
if "evaluation" in results:
    print("\nBERT Score Evaluation:")
    print("=====================")
    for model, scores in results["evaluation"].items():
        if scores:
            print(f"\n{model.capitalize()}:")
            print(f"  Precision: {scores['precision']:.4f}")
            print(f"  Recall: {scores['recall']:.4f}")
            print(f"  F1: {scores['f1']:.4f}")


Getting model rankings...
Sending prompt to Mistral API...
Creating ensemble ranking...
Final ranking...
Ranking 22 abstracts based on relevance to research question...


Some weights of BertModel were not initialized from the model checkpoint at naver/splade-cocondenser-ensembledistil and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Generating model summaries...
Sending prompt to Mistral API...
Creating ensemble summary...
Sending prompt to Mistral API...
Generating model summaries for each paper...
Sending prompt to Mistral API...
Sending prompt to Mistral API...
Sending prompt to Mistral API...
Sending prompt to Mistral API...
Sending prompt to Mistral API...
Sending prompt to Mistral API...
Sending prompt to Mistral API...
Sending prompt to Mistral API...
Sending prompt to Mistral API...
Sending prompt to Mistral API...
Sending prompt to Mistral API...
Sending prompt to Mistral API...
Sending prompt to Mistral API...
Sending prompt to Mistral API...
Sending prompt to Mistral API...
Sending prompt to Mistral API...
Sending prompt to Mistral API...
Sending prompt to Mistral API...
Sending prompt to Mistral API...
Sending prompt to Mistral API...
Sending prompt to Mistral API...
Sending prompt to Mistral API...
Creating ensemble summary for each paper..
Sending prompt to Mistral API...
Sending prompt to Mistral AP

In [ ]:
# reference_summary = """
# Background
# Disasters are becoming more frequent due to the impact of extreme weather events attributed to climate change, causing loss of lives, property, and psychological trauma. Mental health response to disasters emphasizes prevention and mitigation, and mobile health (mHealth) apps have been used for mental health promotion and treatment. However, little is known about their use in the mental health components of disaster management.

# Objective
# This scoping review was conducted to explore the use of mobile phone apps for mental health responses to natural disasters and to identify gaps in the literature.

# Methods
# We identified relevant keywords and subject headings and conducted comprehensive searches in 6 electronic databases. Studies in which participants were exposed to a man-made disaster were included if the sample also included some participants exposed to a natural hazard. Only full-text studies published in English were included. The initial titles and abstracts of the unique papers were screened by 2 independent review authors. Full texts of the selected papers that met the inclusion criteria were reviewed by the 2 independent reviewers. Data were extracted from each selected full-text paper and synthesized using a narrative approach based on the outcome measures, duration, frequency of use of the mobile phone apps, and the outcomes. This scoping review was reported according to the PRISMA-ScR (Preferred Reporting Items for Systematic Reviews and Meta-Analyses extension for Scoping Reviews).

# Results
# Of the 1398 papers retrieved, 5 were included in this review. A total of 3 studies were conducted on participants exposed to psychological stress following a disaster while 2 were for disaster relief workers. The mobile phone apps for the interventions included Training for Life Skills, Sonoma Rises, Headspace, Psychological First Aid, and Substance Abuse and Mental Health Services Administration (SAMHSA) Behavioural Health Disaster Response Apps. The different studies assessed the effectiveness or efficacy of the mobile app, feasibility, acceptability, and characteristics of app use or predictors of use. Different measures were used to assess the effectiveness of the apps’ use as either the primary or secondary outcome.

# Conclusions
# A limited number of studies are exploring the use of mobile phone apps for mental health responses to disasters. The 5 studies included in this review showed promising results. Mobile apps have the potential to provide effective mental health support before, during, and after disasters. However, further research is needed to explore the potential of mobile phone apps in mental health responses to all hazards.

# Keywords: mental health, disasters, mobile health, mHealth, application, applications, app, apps, smartphone, stress, psychological, traumatic, disaster, disasters, hazard, hazards, emergency, psychological trauma, mobile apps, trauma, scoping, review methods, review methodology, mobile phone
# """
# abstracts, titles = get_abstracts_from_papers(search_results)
# results = run_review_pipeline(query, abstracts, titles, reference_summary)

# # Print rankings
# print("\nPaper Rankings:")
# print("==============")
# print("\nGemini Ranking:")
# # print("results!!!----------->", results)
# for i, idx in enumerate(results["rankings"]["gemini"]["ranking"]):
#     print(f"{i+1}. {titles[idx-1]}")

# print("\nMistral Ranking:")
# for i, idx in enumerate(results["rankings"]["mistral"]["ranking"]):
#     print(f"{i+1}. {titles[idx-1]}")

# print("\nEnsemble Ranking:")
# for i, idx in enumerate(results["rankings"]["ensemble"]["ranking"]):
#     print(f"{i+1}. {titles[idx]} (Score: {results['rankings']['ensemble']['scores'][i]:.4f})")

# print("\nFinal Ranking:")
# for i in (results["rankings"]["final"]):
#   print(f"{i['rank']}.{i['title']} (Score: {i['relevance_score']:.4f})")

# # Print summaries (truncated for brevity)
# print("\nConsolidated Summary:")
# print("=================================")
# for model, summary in results["summaries"].items():
#     if summary:
#         print(f"\n{model.capitalize()} Summary: {summary[:]}...")

# # Print BERT Score evaluation
# if "evaluation" in results:
#     print("\nBERT Score Evaluation:")
#     print("=====================")
#     for model, scores in results["evaluation"].items():
#         if scores:
#             print(f"\n{model.capitalize()}:")
#             print(f"  Precision: {scores['precision']:.4f}")
#             print(f"  Recall: {scores['recall']:.4f}")
#             print(f"  F1: {scores['f1']:.4f}")
